In [35]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
from scipy.ndimage import interpolation as inter

In [36]:
def load_and_binarize(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Invert+Otsu so writing is white on black
    th = cv2.threshold(gray, 0, 255,
                       cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    return th

In [37]:
def deskew(img_bin):
    coords = np.column_stack(np.where(img_bin > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = 90 + angle
    else:
        angle = angle

    (h, w) = img_bin.shape  # <-- Moved here

    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    return cv2.warpAffine(img_bin, M, (w, h),
                          flags=cv2.INTER_CUBIC,
                          borderMode=cv2.BORDER_CONSTANT,
                          borderValue=0)


In [55]:
def segment_lines(img_bin, min_height=10):
    if img_bin is None:
        raise ValueError("Input image is None")
    
    # Ensure grayscale
    if len(img_bin.shape) == 3:
        img_bin = cv2.cvtColor(img_bin, cv2.COLOR_BGR2GRAY)
    
    # Binarize to 0/255 uint8
    img_bin = (img_bin > 128).astype(np.uint8) * 255

    # Replace OpenCV reduce with NumPy sum
    hist = np.sum(img_bin, axis=1)

    lines = []
    in_line = False
    start = 0
    for i, val in enumerate(hist):
        if val > 0 and not in_line:
            in_line = True
            start = i
        elif val == 0 and in_line:
            in_line = False
            if i - start > min_height:
                lines.append((start, i))
    return lines


In [56]:
def segment_chars(line_img, min_w=5, min_h=5):
    # Find contours in each line
    cnts, _ = cv2.findContours(line_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    chars = []
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        if w >= min_w and h >= min_h:
            chars.append((x,y,w,h))
    # Sort left-to-right
    chars = sorted(chars, key=lambda b: b[0])
    return chars

In [57]:
def extract_strokes(char_img):
    # Normalize to 0/1
    bin_char = (char_img > 0).astype(np.uint8)
    # Skeletonize
    skel = skeletonize(bin_char).astype(np.uint8) * 255
    return skel

In [58]:
def preprocess_page(path):
    # 1. Load & binarize
    bin_page = load_and_binarize(path)
    # 2. Deskew
    deskewed = deskew(bin_page)
    # 3. Line segmentation
    lines = segment_lines(deskewed)
    char_images = []
    for (y0, y1) in lines:
        line_img = deskewed[y0:y1, :]
        # 4. Character segmentation
        for (x,y,w,h) in segment_chars(line_img):
            char_crop = line_img[y:y+h, x:x+w]
            # 5. Resize to model size (e.g. 28×28) and pad
            h0, w0 = char_crop.shape
            scale = 28.0 / max(h0, w0)
            resized = cv2.resize(char_crop, None, fx=scale, fy=scale,
                                 interpolation=cv2.INTER_AREA)
            # center in 28×28
            canvas = np.zeros((28,28), dtype=np.uint8)
            dx = (28 - resized.shape[1])//2
            dy = (28 - resized.shape[0])//2
            canvas[dy:dy+resized.shape[0], dx:dx+resized.shape[1]] = resized
            # 6. Stroke extraction
            skel = extract_strokes(canvas)
            char_images.append(skel)
    return char_images

In [60]:
def filter_valid_boxes(boxes, min_area=20):
    """Filter out character boxes that are too small (likely noise)."""
    filtered = []
    for (x0, y0, x1, y1) in boxes:
        area = (x1 - x0) * (y1 - y0)
        if area >= min_area and (x1 - x0) > 1 and (y1 - y0) > 1:
            filtered.append((x0, y0, x1, y1))
    return filtered


In [59]:
if __name__ == "__main__":
    page_path = r"D:\STUDY\Sumit\lectures sumit\college\sem 6\test.jpeg"
    chars = preprocess_page(page_path)
    # Example: save out to disk
    for i, ch in enumerate(chars):
        cv2.imwrite(f"char_{i:03d}.png", ch)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4058: error: (-215:Assertion failed) !dsize.empty() in function 'cv::resize'


In [49]:
print("Image shape:", img_bin.shape)
print("Image dtype:", img_bin.dtype)
print("Unique values:", np.unique(img_bin))


NameError: name 'img_bin' is not defined